<a href="https://colab.research.google.com/github/405620294/webcrawler_Tmall/blob/main/%E7%B0%A1%E5%96%AE%E8%B2%9D%E6%B0%8F_%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
raw_df = pd.read_excel('comment.xlsx', ) 
raw_df.isna().sum() # 算出各欄位有多少缺失值 # 2716 target缺失

data_df = raw_df[raw_df['等級'].isna() == False]

In [ ]:
# 資料切片

import numpy as np
from sklearn.model_selection import train_test_split
train_df, test_df =  train_test_split(data_df,test_size = 0.1)

In [ ]:
# step3. 準備y_train,y_test >> 替換(文字→數字) 

data_df['等級'].value_counts() # 算出獨特值出現的次數 好1468 差744 中64
u = data_df['等級'].unique() # list
sco2num = {name:i for i, name in enumerate(u)}

y_train = train_df['等級'].replace(sco2num)
y_test = test_df['等級'].replace(sco2num)

In [ ]:
# step4. 準備x_train,x_test >> 分詞 

import jieba.analyse
from urllib.request import urlretrieve
import jieba
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
jieba.load_userdict("自定義.txt") #使用者自定義辭典的路徑

def poemcut(p):
  return ' '.join(jieba.cut(p)) # 在分詞中間插入空白

x_train = train_df['評論'].apply(poemcut)
x_test = test_df['評論'].apply(poemcut)

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.778 seconds.
Prefix dict has been built successfully.


2964    大家 别买 了 哈 ！ ！ 是 假货 ， 看 图 上面 的 是 本店 的 假货 ， 下面 的...
2845                                    颜色 超 好看   是 我 的 菜
982                                      朋友 推荐 的 买 的 ， 好用
239                                       这个 颜色 好看 ZIGGIE
4050    超级 好看 ， 买 了 一支 来 试一试 颜色 ， 感觉 太 好看 了 ， 还 想 再 购入...
                              ...                        
121     想 买 一只 遮住 唇色 的 ， 这 一款 看见 小红书 很多 推荐 ， 以为 可以 遮 一...
4847                                原来 是 假货   怪不得 那么 不好 用
395     整体 评价 ： 针 不 戳   保湿 效果 ： 还 行   唇膏 质地 ： 厚实 细腻   ...
2180        这家 店 是 假 的 ， 谁 知道 某某 书上 是 正版 渠道 呢 ！ 真的 好 失望 ！
3268    颜色 还 行 ， 但是 听说 淘宝 卡泡 旗舰店 是 假 的 ， 确实 和 我 以前 买 的...
Name: 評論, Length: 2048, dtype: object

In [ ]:
# step5. 找出有多少詞彙(欄位) & 出現的次數 >>　稀疏矩陣

from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train) # 學習&轉換
x_test_count = vec.transform(x_test) # 轉換

# vec.vocabulary_ # 字典 每個欄位代表的詞彙

<2048x2948 sparse matrix of type '<class 'numpy.int64'>'
	with 25044 stored elements in Compressed Sparse Row format>

In [ ]:
# step6 多項式單純貝氏分類器 Multinomial Naive Bayes >> 同樣的東西出現多次，可以直接平方(可以非整數)
# x_train,y_train
# x_test,y_test

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count, y_train) # MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) >> 參數

pre = clf.predict(x_test_count)
from sklearn.metrics import accuracy_score
accuracy_score(pre, y_test) # 0.9078947368421053

In [ ]:
# step7 調整參數，優化模型　>> smoothing

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1)
clf.fit(x_train_count, y_train) 

pre = clf.predict(x_test_count)
from sklearn.metrics import accuracy_score
accuracy_score(pre, y_test) # 0.9035087719298246

0.9078947368421053

In [ ]:
# step8 最容易被誤判的地方 >>　混淆矩陣 confusion_matrix
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
      index = [name + '(原本)' for name in u ],
      columns = [name + '(預測)' for name in u ])

,好(預測),差(預測),中(預測)
好(原本),128,4,0
差(原本),9,79,0
中(原本),6,2,0


In [ ]:
# step９ 使用者互動
pre_df = raw_df[raw_df['等級'].isna()]
x = pre_df['評論'].apply(poemcut) # 分詞
x_count = vec.transform(x) # 詞彙統計

prob = clf.predict_proba(x_count)
for c in prob:
  for i in range(len(u)):
      print(u[i], "的機率:", round(c[i], 4))
pre_df['等級']

串流輸出內容已截斷至最後 5000 行。
差 的機率: 0.0
中 的機率: 0.0
好 的機率: 1.0
差 的機率: 0.0
中 的機率: 0.0
好 的機率: 0.9998
差 的機率: 0.0002
中 的機率: 0.0
好 的機率: 0.9317
差 的機率: 0.0561
中 的機率: 0.0121
好 的機率: 1.0
差 的機率: 0.0
中 的機率: 0.0
好 的機率: 0.9999
差 的機率: 0.0001
中 的機率: 0.0
好 的機率: 0.9816
差 的機率: 0.0121
中 的機率: 0.0063
好 的機率: 0.9959
差 的機率: 0.0023
中 的機率: 0.0018
好 的機率: 1.0
差 的機率: 0.0
中 的機率: 0.0
好 的機率: 0.9994
差 的機率: 0.0006
中 的機率: 0.0
好 的機率: 0.9996
差 的機率: 0.0004
中 的機率: 0.0
好 的機率: 0.9698
差 的機率: 0.0285
中 的機率: 0.0017
好 的機率: 0.0524
差 的機率: 0.9463
中 的機率: 0.0013
好 的機率: 0.6656
差 的機率: 0.3035
中 的機率: 0.0308
好 的機率: 1.0
差 的機率: 0.0
中 的機率: 0.0
好 的機率: 0.0973
差 的機率: 0.8999
中 的機率: 0.0028
好 的機率: 0.9917
差 的機率: 0.0081
中 的機率: 0.0002
好 的機率: 0.8844
差 的機率: 0.1064
中 的機率: 0.0092
好 的機率: 1.0
差 的機率: 0.0
中 的機率: 0.0
好 的機率: 1.0
差 的機率: 0.0
中 的機率: 0.0
好 的機率: 0.8435
差 的機率: 0.1562
中 的機率: 0.0003
好 的機率: 0.9619
差 的機率: 0.0348
中 的機率: 0.0032
好 的機率: 1.0
差 的機率: 0.0
中 的機率: 0.0
好 的機率: 0.9957
差 的機率: 0.0043
中 的機率: 0.0
好 的機率: 0.3234
差 的機率: 0.6746
中 的機率: 0.002
好 的機率: 0.0413
差 的機率: 0.9299
